In [1]:
from llama_index import ListIndex, LLMPredictor
from llama_index.llms import OpenAI
from llama_index.indices.composability import ComposableGraph

ModuleNotFoundError: No module named 'llama_index'

In [2]:
# set summary text for each doc
index_summaries = [f"Class materials for academic {year}" for year in years]

NameError: name 'years' is not defined

In [3]:
# set number of output tokens
# llm = OpenAI(temperature=0, max_tokens=512, model="gpt-4")
# service_context = ServiceContext.from_defaults(llm=llm, chunk_size=512)

# # initialize simple vector indices + global vector index
# # NOTE: don't run this cell if the indices are already loaded!
# index_set = {}
# for year in years:
#     cur_index = VectorStoreIndex.from_documents(
#         doc_set[year], service_context=service_context
#     )
#     index_set[year] = cur_index

NameError: name 'OpenAI' is not defined

In [4]:
# define a list index over the vector indices
# allows us to synthesize information across each index
graph = ComposableGraph.from_indices(
    ListIndex,
    [index_set[y] for y in years],
    index_summaries=index_summaries,
    service_context=service_context,
)

NameError: name 'ComposableGraph' is not defined

## Setting up the Chatbot Agent

We use Langchain to define the outer chatbot abstraction. We use LlamaIndex as a core Tool within this abstraction.

In [5]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import initialize_agent

from llama_index.langchain_helpers.agents import (
    LlamaToolkit,
    create_llama_chat_agent,
    IndexToolConfig,
)

ModuleNotFoundError: No module named 'langchain'

In [6]:
# define a decompose transform
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine

decompose_transform = DecomposeQueryTransform(
    service_context.llm_predictor, verbose=True
)

# define custom query engines
custom_query_engines = {}
for index in index_set.values():
    query_engine = index.as_query_engine()
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_metadata={"index_summary": index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine
custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    response_mode="tree_summarize",
    verbose=True,
)

# construct query engine
graph_query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)

ModuleNotFoundError: No module named 'llama_index'

In [7]:
# index configs
index_configs = []
for y in range(1):
    query_engine = index_set[y].as_query_engine(
        similarity_top_k=3,
    )
    tool_config = IndexToolConfig(
        query_engine=query_engine,
        name=f"Vector Index {y}",
        description=f"useful for when you want to answer queries about the {y} SEC 10-K for Uber",
        tool_kwargs={"return_direct": True, "return_sources": True},
    )
    index_configs.append(tool_config)

# graph config
graph_config = IndexToolConfig(
    query_engine=graph_query_engine,
    name=f"Graph Index",
    description="useful for when you want to answer queries that require analyzing multiple SEC 10-K documents for Uber.",
    tool_kwargs={"return_direct": True, "return_sources": True},
    return_sources=True,
)

toolkit = LlamaToolkit(index_configs=index_configs, graph_configs=[graph_config])

NameError: name 'index_set' is not defined

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm = OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(toolkit, llm, memory=memory, verbose=True)

NameError: name 'ConversationBufferMemory' is not defined

In [9]:
agent_chain.run(input="hi, i am bob")

NameError: name 'agent_chain' is not defined

In [10]:
agent_chain.run(input="What were some of the biggest risk factors in 2020 for Uber?")

NameError: name 'agent_chain' is not defined

In [11]:
cross_query_str = "Compare/contrast the risk factors described in the Uber 10-K across years. Give answer in bullet points."

response = agent_chain.run(input=cross_query_str)

NameError: name 'agent_chain' is not defined

In [12]:
# parse the response w/ sources
import json

response_json = json.loads(response)
print(response)

NameError: name 'response' is not defined

### Setup Chatbot Loop Within Notebook

We'll keep a running loop so that you can converse with the agent. 

In [13]:
# reinitialize agent
memory = ConversationBufferMemory(memory_key="chat_history")
llm = OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
)

NameError: name 'ConversationBufferMemory' is not defined

In [14]:
while True:
    text_input = input("User: ")
    response = agent_chain.run(input=text_input)
    print(f"Agent: {response}")

KeyboardInterrupt: Interrupted by user